# Profitable App Profiles for the App Store and Google Play Markets
The aim is to find the app profiles that are profitable for the App Store and Google Play markets. 

Scenario:

We're working as data analysts for a company that builds Android and iOS mobile apps, and our task is to enable our team of developers to make data-driven decisions with respect to the kind of apps they build.
At our company, we only build apps that are free to download and install, and our main source of revenue consists of in-app ads. This means that our revenue for any given app is mostly influenced by the number of users that use our app. Our goal for this project is to analyze data to help our developers understand what kind of apps are likely to attract more users.

## Data sets used:
* [Google Play store apps data](https://www.kaggle.com/lava18/google-play-store-apps/home) containing data for approximately ten thousand Android apps from Google Play
* [iOS Apps Store apps data](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home) containing data for approximately seven thousand iOS apps from the App Store

In [1]:
from csv import reader

# The Google Play Store data set
opened_file = open("datasets/googleplaystore.csv")
read_file = reader(opened_file)
android_apps_data = list(read_file)
android_header = android_apps_data[0]
android = android_apps_data[1:]

# The App Store data set
opened_file = open("datasets/AppleStore.csv")
read_file = reader(opened_file)
ios_apps_data = list(read_file)
ios_header = ios_apps_data[0]
ios = ios_apps_data[1:]

Defining a handy function `explore_data()` that we can use repeatedly to explore data in amore readable way. Also added an option for our function to show number of rows and columns for any data set. 

In [2]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print("\n") # adds a new (empty) line between rows
        
    if rows_and_columns:
        print("Number of rows:", len(dataset))
        print("Number of columns:", len(dataset[0]))

In [3]:
print(android_header)
print("\n")
explore_data(android, 0, 5, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Eve

We see that Google Play store data set has 10841 apps and 13 columns. At a quick glance, the columns that might be useful for the purpose of our analysis are `'App', 'Category', 'Reviews', 'Installs', 'Type', 'Price'` and `'Genres'`.

The description of these column names can be found in the Google Play Store data set [documentation](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home)

Exploring iOS App Store data set:

In [4]:
print(ios_header)
print("\n")
explore_data(ios, 0, 5, True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


['284035177', 'Pandora - Music & Radio', '130242560', 'USD', '0.0', '1126879', '3594', '4.0', '4.5', '8.4.1', '12+', 'Music', '37', '4', '1', '1']


Number of rows: 7197
Number of columns: 16


We have 7197 iOS apps in this data set, and the columns that seem interesting  are: `'track_name', 'currency', 'price', 'rating_count_tot', 'rating_count_ver'` and `'prime_genre'`.

The description of these column names can be found in the iOS App Store data set [documentation](https://www.kaggle.com/lava18/google-play-store-apps/home)


## Deleting Wrong Data

From the [discussion section](https://www.kaggle.com/lava18/google-play-store-apps/discussion) of Google Play store data set, we find there's an issue with data at index 10472 - [discussion link](https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015).

Printing this row and comparing it to the header and another row that's correct:

In [5]:
print(android[10472])

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


In [6]:
print(android_header)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


In [7]:
print(android[5])

['Paper flowers instructions', 'ART_AND_DESIGN', '4.4', '167', '5.6M', '50,000+', 'Free', '0', 'Everyone', 'Art & Design', 'March 26, 2017', '1.0', '2.3 and up']


Observation: Rating for the app at index 10472 is 19 which is not possible (max rating = 5). Deleting this row:

In [8]:
print(len(android))

10841


In [9]:
del android[10472]

In [10]:
print(len(android))

10840


## Removing Duplicate Entries

Check if Google play apps data set has duplicate entries.

In [11]:
unique_apps = []
duplicate_apps = []
for row in android:
    app_name = row[0]
    if app_name in unique_apps:
        duplicate_apps.append(app_name)
    else:
        unique_apps.append(app_name)

print("No. of duplicate google play apps:", len(duplicate_apps))
print("\n");
print("Examples of duplicate apps:\n", duplicate_apps[:15])

No. of duplicate google play apps: 1181


Examples of duplicate apps:
 ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses', 'HipChat - Chat Built for Teams', 'Xero Accounting Software']


Duplicate entries need to removed as we don't want to count certain apps more than once when we analyze the data. We can can decide randomly which duplicate entries are to removed per app, but a better solution is to keep the entry with maximum number of reviews and remove the rest of the entries for that app.

We observed 1181 duplicate app entries in Google Play dataset. After we remove the duplicates expected length of android data set:

In [12]:
print('Expected length of Google play data set after removing duplicate entries:', len(android) - 1181)

Expected length of Google play data set after removing duplicate entries: 9659


In [13]:
reviews_max = {}
for row in android:
    name = row[0];
    n_reviews = float(row[3])
    if (name in reviews_max) and (reviews_max[name] < n_reviews):
        reviews_max[name] = n_reviews
    elif name not in reviews_max:
        reviews_max[name] = n_reviews

Expected length of `reviews_max` dictionary is the difference between total entries and number of duplicate entries in the data set (1181 as computed previously).

In [14]:
print("Expected length of reviews_max = ", len(android) - 1181)
print("Actual length of reviews_max = ", len(reviews_max))

Expected length of reviews_max =  9659
Actual length of reviews_max =  9659


Now we need to remove duplicates using the reviews_max dictionary.

Procedure:
* Create empty list `android_clean` that will reflect our data set after removing all the duplicates
* Create variable `already_added` to keep track of apps that we have already added to `android_clean`
* Loop through the Google Play data set `android`, and for each iteration:
    * Check if no. of reviews for that row is equal to the maximum review for that app using the dictionary `reviews_max` and app should not be in the `already_added` list. We need this second check as it's possible an app can have multiple entries in our data set such that two or more of those entries have no. of reviews same as max review. Without the second check, we would end up with duplicate entries for some apps.
    
    If this condition is satisfied, append this row to `android_clean` list.
    Also, append the app name to `already_added` list.

In [15]:
android_clean = []
already_added = []

In [16]:
for row in android:
    name = row[0]
    n_reviews = float(row[3])
    if (n_reviews == reviews_max[name]) and (name not in already_added):
        android_clean.append(row)
        already_added.append(name)

We explore android_clean and confirm no. of entries is indeed 9659:

In [17]:
explore_data(android_clean, 0, 5, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']


['Paper flowers instructions', 'ART_AND_DESIGN', '4.4', '167', '5.6M', '50,000+', 'Free', '0', 'Everyone', 'Art & Design', 'March 26, 2017', '1.0', '2.3 and up']


Number of rows: 9659
Number of columns: 13


We have 9659 rows, just as expected. 

## Removing non-English apps

The numbers corresponding to the characters we commonly use in an English text are all in the range 0 to 127, according to the ASCII system. Based on this number range, we can build a function that detects whether a character belongs to the set of common English characters or not. If the number is equal to or less than 127, then the character belongs to the set of common English characters.

We use the built-in `ord` function to find out the corresponding encoding no. of each character.

In [18]:
def is_english(string):
    for ch in string:
        if ord(ch) > 127:
            return False
    return True

In [19]:
print(is_english("Instagram"))
print(is_english("爱奇艺PPS -《欢乐颂2》电视剧热播"))

True
False


There is possibility of occurence of characters like (™, — (em dash), – (en dash) that falls outside of ASCII range but apps can be misjudged as non-English if we use the above function as it is.
To minimize the impact of data loss, we'll only remove an app if its name has more that three non-ASCII characters.

In [20]:
def is_english(string):
    non_ascii = 0
    for ch in string:
        if ord(ch) > 127:
            non_ascii += 1
    if non_ascii > 3:
        return False
    return True

In [21]:
print(is_english("Docs To Go™ Free Office Suite"))
print(is_english('Instachat 😜'))

True
True


The function is still not perfect, and very few non-English apps might get past our filter, but this seems good enough at this point in our analysis — we shouldn't spend too much time on optimization at this point.

Below, we use the is_english() function to filter out the non-English apps for both data sets:

In [22]:
android_english = []
ios_english = []

for row in android_clean:
    name = row[0]
    if is_english(name):
        android_english.append(row)
        
for row in ios:
    name = row[1]
    if is_english(name):
        ios_english.append(row)

In [23]:
explore_data(android_english, 0, 3, True)
print("\n")
explore_data(ios_english, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9614
Number of columns: 13


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 

We observe that we are left with 9614 Android apps and 6183 iOS apps.

## Isolating the Free Apps

As we mentioned in the introduction, we only build apps that are free to download and install, and our main source of revenue consists of in-app ads. Our data sets contain both free and non-free apps, and we'll need to isolate only the free apps for our analysis. Below, we isolate the free apps for both our data sets.

In [24]:
android_final = [row for row in android_english if row[7] == "0"]

In [25]:
ios_final = [row for row in ios_english if row[4] == "0.0"]

In [26]:
print(len(android_final))
print(len(ios_final))

8864
3222


Thus, we are left with 8864 Android apps 3222 iOS apps for analysis.

As we mentioned in the introduction, our aim is to determine the kinds of apps that are likely to attract more users because our revenue is highly influenced by the number of people using our apps.

To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:

1. Build a minimal Android version of the app, and add it to Google Play.
2. If the app has a good response from users, we then develop it further.
3. If the app is profitable after six months, we also build an iOS version of the app and add it to the App Store.

Because our end goal is to add the app on both the App Store and Google Play, we need to find app profiles that are successful on both markets. For instance, a profile that might work well for both markets might be a productivity app that makes use of gamification.

Let's begin the analysis by getting a sense of the most common genres for each market. For this, we'll build a frequency table for the `prime_genre` column of the App Store data set, and the `Genres` and `Category` columns of the Google Play data set.

Creating two functions we can use to analyze the frequency tables:
    1. `freq_table()` to generate frequency tables that show percentages.
    2. `display_table()` to display the percentages in descending order.

In [27]:
def freq_table(dataset, index):
    table = {}
    for row in dataset:
        value = row[index]
        if value in table:
            table[value] += 1
        else:
            table[value] = 1 
            
    table_percentages = {}
    total = len(dataset)
    
    for key in table:
        table_percentages[key] = (table[key] / total) * 100
        
    return table_percentages
    

In [28]:
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_content = []
    for key in table:
        table_content.append((table[key], key))
        
    table_content.sort(reverse=True)
    for item in table_content:
        print(item[1], ": ", item[0])

In [29]:
# just trying an alternate way to sort a dict based on values
def display_table_aliter(dataset, index):
    table = freq_table(dataset, index)
    table_sorted = (sorted(table.items(), key=lambda kv: kv[1], reverse=True)) 
    for item in table_sorted:
        print("{}: {}".format(item[0], item[1]))

Freq-table for `prime_genre` column of App Store data set:

In [30]:
display_table_aliter(ios_final, -5)

Games: 58.16263190564867
Entertainment: 7.883302296710118
Photo & Video: 4.9658597144630665
Education: 3.662321539416512
Social Networking: 3.2898820608317814
Shopping: 2.60707635009311
Utilities: 2.5139664804469275
Sports: 2.1415270018621975
Music: 2.0484171322160147
Health & Fitness: 2.0173805090006205
Productivity: 1.7380509000620732
Lifestyle: 1.5828677839851024
News: 1.3345747982619491
Travel: 1.2414649286157666
Finance: 1.1173184357541899
Weather: 0.8690254500310366
Food & Drink: 0.8069522036002483
Reference: 0.5586592178770949
Business: 0.5276225946617008
Book: 0.4345127250155183
Navigation: 0.186219739292365
Medical: 0.186219739292365
Catalogs: 0.12414649286157665


In [31]:
display_table(ios_final, -5)

Games :  58.16263190564867
Entertainment :  7.883302296710118
Photo & Video :  4.9658597144630665
Education :  3.662321539416512
Social Networking :  3.2898820608317814
Shopping :  2.60707635009311
Utilities :  2.5139664804469275
Sports :  2.1415270018621975
Music :  2.0484171322160147
Health & Fitness :  2.0173805090006205
Productivity :  1.7380509000620732
Lifestyle :  1.5828677839851024
News :  1.3345747982619491
Travel :  1.2414649286157666
Finance :  1.1173184357541899
Weather :  0.8690254500310366
Food & Drink :  0.8069522036002483
Reference :  0.5586592178770949
Business :  0.5276225946617008
Book :  0.4345127250155183
Navigation :  0.186219739292365
Medical :  0.186219739292365
Catalogs :  0.12414649286157665


We can see that among the free English apps, more than a half (58.16%) are games. Entertainment apps are close to 8%, followed by photo and video apps, which are close to 5%. Only 3.66% of the apps are designed for education, followed by social networking apps which amount for 3.29% of the apps in our data set.

The general impression is that App Store (at least the part containing free English apps) is dominated by apps that are designed for fun (games, entertainment, photo and video, social networking, sports, music, etc.), while apps with practical purposes (education, shopping, utilities, productivity, lifestyle, etc.) are more rare. However, the fact that fun apps are the most numerous doesn't also imply that they also have the greatest number of users — the demand might not be the same as the offer.

Freq-table for `Genres` and `Category` column of Google Play data set:

In [32]:
display_table(android_final, -4) # Genre

Tools :  8.449909747292418
Entertainment :  6.069494584837545
Education :  5.347472924187725
Business :  4.591606498194946
Productivity :  3.892148014440433
Lifestyle :  3.892148014440433
Finance :  3.7003610108303246
Medical :  3.531137184115524
Sports :  3.463447653429603
Personalization :  3.3167870036101084
Communication :  3.2378158844765346
Action :  3.1024368231046933
Health & Fitness :  3.0798736462093865
Photography :  2.944494584837545
News & Magazines :  2.7978339350180503
Social :  2.6624548736462095
Travel & Local :  2.3240072202166067
Shopping :  2.2450361010830324
Books & Reference :  2.1435018050541514
Simulation :  2.0419675090252705
Dating :  1.861462093862816
Arcade :  1.8501805054151623
Video Players & Editors :  1.7712093862815883
Casual :  1.7599277978339352
Maps & Navigation :  1.3989169675090252
Food & Drink :  1.2409747292418771
Puzzle :  1.128158844765343
Racing :  0.9927797833935018
Role Playing :  0.9363718411552346
Libraries & Demo :  0.9363718411552346
Aut

In [33]:
display_table(android_final, 1) # Category

FAMILY :  18.907942238267147
GAME :  9.724729241877256
TOOLS :  8.461191335740072
BUSINESS :  4.591606498194946
LIFESTYLE :  3.9034296028880866
PRODUCTIVITY :  3.892148014440433
FINANCE :  3.7003610108303246
MEDICAL :  3.531137184115524
SPORTS :  3.395758122743682
PERSONALIZATION :  3.3167870036101084
COMMUNICATION :  3.2378158844765346
HEALTH_AND_FITNESS :  3.0798736462093865
PHOTOGRAPHY :  2.944494584837545
NEWS_AND_MAGAZINES :  2.7978339350180503
SOCIAL :  2.6624548736462095
TRAVEL_AND_LOCAL :  2.33528880866426
SHOPPING :  2.2450361010830324
BOOKS_AND_REFERENCE :  2.1435018050541514
DATING :  1.861462093862816
VIDEO_PLAYERS :  1.7937725631768955
MAPS_AND_NAVIGATION :  1.3989169675090252
FOOD_AND_DRINK :  1.2409747292418771
EDUCATION :  1.1620036101083033
ENTERTAINMENT :  0.9589350180505415
LIBRARIES_AND_DEMO :  0.9363718411552346
AUTO_AND_VEHICLES :  0.9250902527075812
HOUSE_AND_HOME :  0.8235559566787004
WEATHER :  0.8009927797833934
EVENTS :  0.7107400722021661
PARENTING :  0.6543

The landscape seems significantly different on Google Play: there are not that many apps designed for fun, and it seems that a good number of apps are designed for practical purposes (family, tools, business, lifestyle, productivity, etc.). However, if we investigate this further, we can see that the family category (which accounts for almost 19% of the apps) means mostly games for kids.


`Genres` column is much more granular (it has more categories). We're only looking for the bigger picture at the moment, so we'll only work with the Category column moving forward.

Up to this point, we found that the App Store is dominated by apps designed for fun, while Google Play shows a more balanced landscape of both practical and for-fun apps. Now we'd like to get an idea about the kind of apps that have most users.

## Most Popular Apps by Genre on the App Store

In [34]:
freq_table_genre = freq_table(ios_final, -5)

In [35]:
for genre in freq_table_genre:
    len_genre = 0
    total = 0  # total no. ratings for apps of this genre  
    for row in ios_final:
        genre_app = row[-5]
        if genre_app == genre:
            len_genre += 1
            total += float(row[5])
    avg_no_of_user_ratings = total / len_genre
    print(genre, ": ", avg_no_of_user_ratings)

Social Networking :  71548.34905660378
Photo & Video :  28441.54375
Games :  22788.6696905016
Music :  57326.530303030304
Reference :  74942.11111111111
Health & Fitness :  23298.015384615384
Weather :  52279.892857142855
Utilities :  18684.456790123455
Travel :  28243.8
Shopping :  26919.690476190477
News :  21248.023255813954
Navigation :  86090.33333333333
Lifestyle :  16485.764705882353
Entertainment :  14029.830708661417
Food & Drink :  33333.92307692308
Sports :  23008.898550724636
Book :  39758.5
Finance :  31467.944444444445
Education :  7003.983050847458
Productivity :  21028.410714285714
Business :  7491.117647058823
Catalogs :  4004.0
Medical :  612.0


It seems apps of Navigation genre are most popular among the users but this figure is skewed due to apps like Waze and Google Maps which have considerably high no. of user ratings compared to other apps of this category:

In [36]:
for row in ios_final:
    genre_app = row[-5]
    app_name = row[1]
    if genre_app == "Navigation":
        no_of_user_ratings = row[5]
        print(app_name, ": ", no_of_user_ratings)

Waze - GPS Navigation, Maps & Real-time Traffic :  345046
Google Maps - Navigation & Transit :  154911
Geocaching® :  12811
CoPilot GPS – Car Navigation & Offline Maps :  3582
ImmobilienScout24: Real Estate Search in Germany :  187
Railway Route Search :  5


The same pattern applies to social networking apps, where the average number is heavily influenced by a few giants like Facebook, Pinterest, Skype, etc. Same applies to music apps, where a few big players like Pandora, Spotify, and Shazam heavily influence the average number.

Our aim is to find popular genres, but navigation, social networking or music apps might seem more popular than they really are. The average number of ratings seem to be skewed by very few apps which have hundreds of thousands of user ratings, while the other apps may struggle to get past the 10,000 threshold.

Reference apps category has around 74942 no. of average user ratings. Lets have a look at the distribution of no. of ratings for apps of this category: 

In [37]:
for row in ios_final:
    if row[-5] == "Reference":
        print(row[1], ": ", row[5])

Bible :  985920
Dictionary.com Dictionary & Thesaurus :  200047
Dictionary.com Dictionary & Thesaurus for iPad :  54175
Google Translate :  26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran :  18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition :  17588
Merriam-Webster Dictionary :  16849
Night Sky :  12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) :  8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools :  4693
GUNS MODS for Minecraft PC Edition - Mods Tools :  1497
Guides for Pokémon GO - Pokemon GO News and Cheats :  826
WWDC :  762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free :  718
VPN Express :  14
Real Bike Traffic Rider Virtual Reality Glasses :  8
教えて!goo :  0
Jishokun-Japanese English Dictionary & Translator :  0


Again, averge no. of user ratings for apps of 'Reference' category is skewed by apps like 'Bible' and 'Dictionary.com Dictionary & Thesaurus' but at least other apps are able to reach 10000 ratings. This genre looks promising.

While we observed that App Store is highly saturated by apps of fun category, making an app for reference category makes sense. A practical app might have more of a chance to stand out among the huge number of apps on the App Store. 

## Most Popular Apps by Genre on Google Play 

In the Google Play data set, we have no. of installs for each app. We will use this column for determing most popular app category.

In [38]:
display_table(android_final, 5)

1,000,000+ :  15.726534296028879
100,000+ :  11.552346570397113
10,000,000+ :  10.548285198555957
10,000+ :  10.198555956678701
1,000+ :  8.393501805054152
100+ :  6.915613718411552
5,000,000+ :  6.825361010830325
500,000+ :  5.561823104693141
50,000+ :  4.7721119133574
5,000+ :  4.512635379061372
10+ :  3.5424187725631766
500+ :  3.2490974729241873
50,000,000+ :  2.3014440433213
100,000,000+ :  2.1322202166064983
50+ :  1.917870036101083
5+ :  0.78971119133574
1+ :  0.5076714801444043
500,000,000+ :  0.2707581227436823
1,000,000,000+ :  0.22563176895306858
0+ :  0.04512635379061372
0 :  0.01128158844765343


The installs column values are not precise. Howerver, for our purposes we need to get an overall picture of the popularity of various genre of apps.
So, we will treat these values as it is, like 1000+ can be treated as 1000.

In [39]:
freq_table_android = freq_table(android_final, 1)

In [40]:
for category in freq_table_android:
    len_category = 0;
    total = 0
    for row in android_final:
        category_app = row[1]
        if category_app == category:
            len_category += 1
            # Removing comma and plus characters from no. of installs value
            installs = float(row[5].replace('+', '').replace(',', ""))
            total += float(installs)
    avg_installs = total / len_category
    # Printing app category and avg no. of installs for apps in that category
    print(category, ": ", avg_installs)  

ART_AND_DESIGN :  1986335.0877192982
AUTO_AND_VEHICLES :  647317.8170731707
BEAUTY :  513151.88679245283
BOOKS_AND_REFERENCE :  8767811.894736841
BUSINESS :  1712290.1474201474
COMICS :  817657.2727272727
COMMUNICATION :  38456119.167247385
DATING :  854028.8303030303
EDUCATION :  1833495.145631068
ENTERTAINMENT :  11640705.88235294
EVENTS :  253542.22222222222
FINANCE :  1387692.475609756
FOOD_AND_DRINK :  1924897.7363636363
HEALTH_AND_FITNESS :  4188821.9853479853
HOUSE_AND_HOME :  1331540.5616438356
LIBRARIES_AND_DEMO :  638503.734939759
LIFESTYLE :  1437816.2687861272
GAME :  15588015.603248259
FAMILY :  3695641.8198090694
MEDICAL :  120550.61980830671
SOCIAL :  23253652.127118643
SHOPPING :  7036877.311557789
PHOTOGRAPHY :  17840110.40229885
SPORTS :  3638640.1428571427
TRAVEL_AND_LOCAL :  13984077.710144928
TOOLS :  10801391.298666667
PERSONALIZATION :  5201482.6122448975
PRODUCTIVITY :  16787331.344927534
PARENTING :  542603.6206896552
WEATHER :  5074486.197183099
VIDEO_PLAYERS 

It seems 'COMMUNICATION' category apps are most popular.
Looking closely at no. of installs for these apps:



In [41]:
for row in android_final:
    if row[1] == "COMMUNICATION":
        print(row[0], ": ", row[5])

WhatsApp Messenger :  1,000,000,000+
Messenger for SMS :  10,000,000+
My Tele2 :  5,000,000+
imo beta free calls and text :  100,000,000+
Contacts :  50,000,000+
Call Free – Free Call :  5,000,000+
Web Browser & Explorer :  5,000,000+
Browser 4G :  10,000,000+
MegaFon Dashboard :  10,000,000+
ZenUI Dialer & Contacts :  10,000,000+
Cricket Visual Voicemail :  10,000,000+
TracFone My Account :  1,000,000+
Xperia Link™ :  10,000,000+
TouchPal Keyboard - Fun Emoji & Android Keyboard :  10,000,000+
Skype Lite - Free Video Call & Chat :  5,000,000+
My magenta :  1,000,000+
Android Messages :  100,000,000+
Google Duo - High Quality Video Calls :  500,000,000+
Seznam.cz :  1,000,000+
Antillean Gold Telegram (original version) :  100,000+
AT&T Visual Voicemail :  10,000,000+
GMX Mail :  10,000,000+
Omlet Chat :  10,000,000+
My Vodacom SA :  5,000,000+
Microsoft Edge :  5,000,000+
Messenger – Text and Video Chat for Free :  1,000,000,000+
imo free video calls and chat :  500,000,000+
Calls & Tex

Goodbox - Mega App :  100,000+
Call Blocker - Blacklist, SMS Blocker :  1,000,000+
[EF]ShoutBox :  100+
Eg Call :  10,000+
ei :  10+
EJ messenger :  10+
Ek IRA :  10+
Orfox: Tor Browser for Android :  10,000,000+
EO Mumbai :  10+
EP RSS Reader :  100+
Voxer Walkie Talkie Messenger :  10,000,000+
ES-1 :  500+
Hangouts Dialer - Call Phones :  10,000,000+
EU Council :  1,000+
Council Voting Calculator :  5,000+
Have your say on Europe :  500+
Programi podrške EU :  100+
Inbox.eu :  10,000+
Web Browser for Android :  1,000,000+
Everbridge :  100,000+
Best Auto Call Recorder Free :  500+
EZ Wifi Notification :  10,000+
Test Server SMS FA :  5+
Lite for Facebook Messenger :  1,000,000+
FC Browser - Focus Privacy Browser :  1,000+
EHiN-FH conferenceapp :  100+
Carpooling FH Hagenberg :  100+
Wi-Fi Auto-connect :  1,000,000+
Talkie - Wi-Fi Calling, Chats, File Sharing :  500,000+
WeFi - Free Fast WiFi Connect & Find Wi-Fi Map :  1,000,000+
Sat-Fi :  5,000+
Portable Wi-Fi hotspot Free :  100,00

Again, we observe that average no. of installs for apps of "COMMUNICATION" category is skewed by apps like WhatsApp, Facebook Messenger, Skype, Google Chrome, Gmail, and Hangouts.

The books and reference genre looks fairly popular as well, with an average number of installs of 8,767,811. It's interesting to explore this in more depth, since we found this genre has some potential to work well on the App Store, and our aim is to recommend an app genre that shows potential for being profitable on both the App Store and Google Play.

Let's take a look at some of the apps from this genre and their number of installs:

In [42]:
for row in android_final:
    if row[1] == "BOOKS_AND_REFERENCE":
        print(row[0], ": ", row[5])

E-Book Read - Read Book for free :  50,000+
Download free book with green book :  100,000+
Wikipedia :  10,000,000+
Cool Reader :  10,000,000+
Free Panda Radio Music :  100,000+
Book store :  1,000,000+
FBReader: Favorite Book Reader :  10,000,000+
English Grammar Complete Handbook :  500,000+
Free Books - Spirit Fanfiction and Stories :  1,000,000+
Google Play Books :  1,000,000,000+
AlReader -any text book reader :  5,000,000+
Offline English Dictionary :  100,000+
Offline: English to Tagalog Dictionary :  500,000+
FamilySearch Tree :  1,000,000+
Cloud of Books :  1,000,000+
Recipes of Prophetic Medicine for free :  500,000+
ReadEra – free ebook reader :  1,000,000+
Anonymous caller detection :  10,000+
Ebook Reader :  5,000,000+
Litnet - E-books :  100,000+
Read books online :  5,000,000+
English to Urdu Dictionary :  500,000+
eBoox: book reader fb2 epub zip :  1,000,000+
English Persian Dictionary :  500,000+
Flybook :  500,000+
All Maths Formulas :  1,000,000+
Ancestry :  5,000,00

Still, there are apps that seem to skew average no. of installs:

In [43]:
for row in android_final:
    if row[1] == "BOOKS_AND_REFERENCE" and (row[5] == '1,000,000,000+'
                                            or row[5] == '500,000,000+'
                                            or row[5] == '100,000,000+'):
        print(row[0], ": ", row[5])

Google Play Books :  1,000,000,000+
Bible :  100,000,000+
Amazon Kindle :  100,000,000+
Wattpad 📖 Free Books :  100,000,000+
Audiobooks from Audible :  100,000,000+


As these are very few, developing apps in this category can be a good deccision.

This niche seems to be dominated by reading ebooks, as well as various collections of libraries and dictionaries, so it's probably not a good idea to build similar apps since there'll be some significant competition.